In [1]:
from datetime import datetime

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import os
import json
import pandas as pd
import numpy as np

import matplotlib as mlp
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname = "C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family = font_name)
mlp.rcParams["axes.unicode_minus"] = False

#### 전처리

In [4]:
[라벨]토마토_정상_train.csv
[라벨]토마토_정상_train(leaf).csv

[라벨]토마토_질병_train.csv

[라벨]토마토_증강_train.csv


[라벨]토마토_정상_val.csv

[라벨]토마토_증강_val.csv

[라벨]토마토_질병_val.csv

In [117]:
# yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_train.csv")
# yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_val.csv")
# yolo_df

In [118]:
# yolo_df.area.value_counts()

In [115]:
# yolo_df[yolo_df["area"] == 1] # 열매

# yolo_df[yolo_df["area"] == 2] # 꽃

# yolo_df[yolo_df["area"] == 3] # 잎

# yolo_df[yolo_df["area"] == 5] # 줄기

### 정상 데이터 area=3 잎

- train
    - 1 열매  6386
    - 3 잎    **4472**
    - 2 꽃    1677
    - 5 줄기   973
    
    
- val
    - 5    2336
    - 3    **1888**
    - 1     101
    - 2      98

In [114]:
# 6386+1677+973,/
# 2336+101+98

# yolo_df_leaf = yolo_df[yolo_df["area"].isin([3])]

# yolo_df_leaf.to_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_train(leaf).csv", encoding="utf-8-sig", index=False)



# fine_train_0 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_train.csv")

# len(list(fine_train_0[~fine_train_0["area"].isin([3])].image))



# fine_val_0 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_val.csv")

# len(list(fine_val_0[~fine_val_0["area"].isin([3])].image))
# fine_val_0[fine_val_0["area"].isin([3])]
# fine_val_0[fine_val_0["area"].isin([3])].to_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_val(leaf).csv", encoding="utf-8-sig", index=False)

# move_no_leaf_list = list(fine_train_0[~fine_train_0["area"].isin([3])].image) + list(fine_val_0[~fine_val_0["area"].isin([3])].image)

# move_no_leaf_list

# no_leaf_df = pd.DataFrame(move_no_leaf_list, columns=["image"])

# no_leaf_df.to_csv("data/tomato_label/tomato_set/no_leaf_names.csv", encoding="utf-8-sig", index=False)

### 7 classes

- 0 정상
<br><br>
- 1 곰팡 초기
- 2 곰팡 중기
- 3 곰팡 말기
<br><br>
- 4 말림 초기
- 5 말림 중기
- 6 말림 말기

### 전처리 완료 csv

In [4]:
# [라벨]토마토_정상_train.csv
[라벨]토마토_정상_train(leaf).csv

[라벨]토마토_질병_train.csv

[라벨]토마토_증강_train.csv


# [라벨]토마토_정상_val.csv
[라벨]토마토_정상_val(leaf).csv

[라벨]토마토_증강_val.csv

[라벨]토마토_질병_val.csv

In [2]:
# yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_train(leaf).csv")
# yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_질병_train.csv")
yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_증강_train.csv")



# yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_val(leaf).csv")
# yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_질병_val.csv")
# yolo_df = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_증강_val.csv")

In [3]:
yolo_df

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,4032,3024,77,1,4,18,11,3,13,1,908,712,3104,2108
1,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,4032,3024,77,1,4,18,11,3,13,1,927,916,3124,2312
2,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,18,11,3,13,1,916,908,2312,3104
3,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,18,11,3,13,1,916,908,2312,3104
4,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,18,11,3,13,1,916,927,2312,3124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19005,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0005...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,644,438,3636,2526
19006,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0006...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,644,497,3636,2586
19007,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0007...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,644,497,3636,2586
19008,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0008...,2020/11/30,3024,4032,77,1,17,19,11,3,12,3,497,396,2586,3388


In [4]:
# yolo_df.disease.value_counts()

In [5]:
yolo_df["disease"] = yolo_df["disease"].replace({18: 1, 19: 2})

In [6]:
yolo_df

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,4032,3024,77,1,4,1,11,3,13,1,908,712,3104,2108
1,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,4032,3024,77,1,4,1,11,3,13,1,927,916,3124,2312
2,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,1,11,3,13,1,916,908,2312,3104
3,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,1,11,3,13,1,916,908,2312,3104
4,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,1,11,3,13,1,916,927,2312,3124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19005,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0005...,2020/11/30,4032,3024,77,1,17,2,11,3,12,3,644,438,3636,2526
19006,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0006...,2020/11/30,4032,3024,77,1,17,2,11,3,12,3,644,497,3636,2586
19007,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0007...,2020/11/30,4032,3024,77,1,17,2,11,3,12,3,644,497,3636,2586
19008,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0008...,2020/11/30,3024,4032,77,1,17,2,11,3,12,3,497,396,2586,3388


In [7]:
## YOLO 기준 컬럼 추가
### cx, cy, w, h

yolo_df["cx"] = (yolo_df["xtl"] + yolo_df["xbr"])/2
yolo_df["cy"] = (yolo_df["ytl"] + yolo_df["ybr"])/2
yolo_df["w"] = yolo_df["xbr"] - yolo_df["xtl"]
yolo_df["h"] = yolo_df["ybr"] - yolo_df["ytl"]

## BB 좌표 비정상 조건 확인
## 0이 있으면 안되는 경우

print("xbr=0 인 경우")
print(yolo_df[yolo_df.xbr == 0])
print("ybr=0 인 경우")
print(yolo_df[yolo_df.ybr == 0])


##  조건 확인
### xbr > xtl
print("xbr > xtl 아닌 경우")
print(yolo_df[yolo_df.xbr < yolo_df.xtl])
### ybr > ytl 
print("ybr > ytl 아닌 경우")
print(yolo_df[yolo_df.ybr < yolo_df.ytl])

## cx, cy, w, h 이미지 크기 기준, 비율로 정규화
### 0~1
yolo_df["cx"] = yolo_df["cx"]/yolo_df["width"]
yolo_df["cy"] = yolo_df["cy"]/yolo_df["height"]
yolo_df["w"] = yolo_df["w"]/yolo_df["width"]
yolo_df["h"] = yolo_df["h"]/yolo_df["height"]

## cx cy w h, 0~1 사이 값인지 확인

print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["cx"] >= 0) & (yolo_df["cx"] <= 1)].cx))
print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["cy"] >= 0) & (yolo_df["cy"] <= 1)].cy))
print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["w"] >= 0) & (yolo_df["w"] <= 1)].w))
print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["h"] >= 0) & (yolo_df["h"] <= 1)].h))


## 0~1 아닌 것 조회
# 0보다 작은 경우가... 설마 있을까?
print("0보다 작은 경우")
print(yolo_df[(yolo_df["cx"] < 0)].cx)
print(yolo_df[(yolo_df["cy"] < 0)].cy)
print(yolo_df[(yolo_df["w"] < 0)].w)
print(yolo_df[(yolo_df["h"] < 0)].h)

print("1보다 큰 경우")
print(yolo_df[(yolo_df["cx"] > 1)])
print(yolo_df[(yolo_df["cy"] > 1)])
print(yolo_df[(yolo_df["w"] > 1)])
print(yolo_df[(yolo_df["h"] > 1)])

## 컬럼 정리, copy
yolo_df.head(0)
yolo_data = yolo_df.copy()
yolo_data = yolo_data.reindex(columns=["disease", "cx", "cy", "w", "h"])
yolo_data.head(0)

## 타입 => 문자열
yolo_data = yolo_data.astype("str")

xbr=0 인 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, cx, cy, w, h]
Index: []
ybr=0 인 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, cx, cy, w, h]
Index: []
xbr > xtl 아닌 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, cx, cy, w, h]
Index: []
ybr > ytl 아닌 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, cx, cy, w, h]
Index: []
(df row수) - (0~1 사이인 row수)/n => 0
(df row수) - (0~1 사이인 row수)/n => 0
(df row수) - (0~1 사이인 row수)/n => 0
(df row수) - (0~1 사이인 row수)/n => 0
0보다 작은 경우
Series([], Name: cx, dtype: float64)
Series([], Name: cy, dtype: float64)
Series([], Name: w, dtype: float64)
Series([], Name: h, dtype: float64)
1보다 큰 경우
Empty DataFrame
Columns: [image, date, wi

In [8]:
yolo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19010 entries, 0 to 19009
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   disease  19010 non-null  object
 1   cx       19010 non-null  object
 2   cy       19010 non-null  object
 3   w        19010 non-null  object
 4   h        19010 non-null  object
dtypes: object(5)
memory usage: 742.7+ KB


In [9]:
yolo_data

,disease,cx,cy,w,h
0,1,0.4975198412698413,0.4662698412698413,0.5446428571428571,0.46164021164021163
1,1,0.5023561507936508,0.5337301587301587,0.544890873015873,0.46164021164021163
2,1,0.5337301587301587,0.4975198412698413,0.46164021164021163,0.5446428571428571
3,1,0.5337301587301587,0.4975198412698413,0.46164021164021163,0.5446428571428571
4,1,0.5337301587301587,0.5023561507936508,0.46164021164021163,0.544890873015873
...,...,...,...,...,...
19005,2,0.5307539682539683,0.49007936507936506,0.7420634920634921,0.6904761904761905
19006,2,0.5307539682539683,0.509755291005291,0.7420634920634921,0.6908068783068783
19007,2,0.5307539682539683,0.509755291005291,0.7420634920634921,0.6908068783068783
19008,2,0.509755291005291,0.46924603174603174,0.6908068783068783,0.7420634920634921


## .json -> .txt (YOLO)

In [234]:
# 개수 한번 더 확인
len(yolo_df.image)

2800

In [235]:
# 저장 경로 확인

num_rows = len(yolo_data)

for i in tqdm(range(num_rows)):
# for i in range(4):
    value = " ".join(yolo_data.iloc[i])

    # 파일 저장 위치!!
    filename = f"data/tomato_yolo/val/labels/{yolo_df['image'][i].split('.')[0]}.txt"
    
    with open(filename, "w") as f:
        f.write(value)
        

num_files = len(os.listdir("data/tomato_yolo/val/labels"))
print(f"경로 내 총 파일 수 : {num_files}")

100%|████████████████████████████████████████████████████████████████████████████| 2800/2800 [00:01<00:00, 2415.20it/s]

경로 내 총 파일 수 : 4934


## 사진 이동 / shuffle

In [156]:
move_df = pd.read_csv("data/tomato_label/tomato_set/move_list_names.csv")
leaf_df = pd.read_csv("data/tomato_label/tomato_set/no_leaf_names.csv")

In [ ]:
C:\final_project\tomato_yolo_set

In [162]:
move_list = list(move_df.image)

In [163]:
leaf_list = list(leaf_df.image)

In [164]:
move_out_list = move_list + leaf_list

In [167]:
import os
import shutil

file_list = move_out_list
# 이동시키고자 하는 파일들의 리스트

# 원본 폴더 경로
# source_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/train/images"
source_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/val/images"

# 대상 폴더 경로
# target_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/move_out/train_out"
target_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/move_out/val_out"

for file_name in file_list:
    # 파일의 전체 경로 생성
    source_file_path = os.path.join(source_folder, file_name)
    
    # 파일 존재 여부 확인
    if os.path.exists(source_file_path):
        # 대상 폴더에 파일 이동
        shutil.move(source_file_path, target_folder)
        print(f'{file_name}을(를) 이동했습니다.')
    else:
        print(f'{file_name}을(를) 찾을 수 없습니다.')

V006_77_0_00_11_01_13_0_c07_20201202_0068_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199.jpg을(를) 찾을 수 없습니다.
V006_77_1_19_11_03_12_1_4997q_20201130_22.jpeg을(를) 찾을 수 없습니다.
V006_77_1_19_11_03_12_3_4521q_20201209_21.jpeg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0000.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0001.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0002.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0003.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0004.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0005.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0006.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0007.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0008.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0009.jpg을(를) 찾을 수 없습니다.
V006_77_1_19_11_03_12_3_4521q_20201209_21_a0000.jpeg을(를) 찾을 수 없습니다.

## 크기 조절

In [23]:
# # train
# input_directory = "C:/juna27/PythonWork/AI/data/tomato_yolo/train/images/"

# output_directory = "C:/juna27/PythonWork/AI/data/tomato_yolo_resize/train/images/"

In [24]:
# # 입력 디렉토리 내의 모든 파일에 대해 반복
# for filename in os.listdir(input_directory):
#     # 파일의 전체 경로
#     input_image_path = os.path.join(input_directory, filename)
    
#     # 이미지 읽어오기
#     input_image = cv2.imread(input_image_path)

#     # 이미지의 너비, 폭 가져오기
#     width = input_image.shape[1]
#     height = input_image.shape[0]

#     # 이미지의 크기 조절
#     output_image = cv2.resize(input_image, dsize=(width // 4, height // 4))

#     # 복사된 이미지의 저장 경로
#     output_image_path = os.path.join(output_directory, filename)

#     # 이미지 저장
#     cv2.imwrite(output_image_path, output_image)

In [27]:
# # val
# input_directory = "C:/juna27/PythonWork/AI/data/tomato_yolo/val/images/"

# output_directory = "C:/juna27/PythonWork/AI/data/tomato_yolo_resize/val/images/"

In [28]:
# # 입력 디렉토리 내의 모든 파일에 대해 반복
# for filename in os.listdir(input_directory):
#     # 파일의 전체 경로
#     input_image_path = os.path.join(input_directory, filename)
    
#     # 이미지 읽어오기
#     input_image = cv2.imread(input_image_path)

#     # 이미지의 너비, 폭 가져오기
#     width = input_image.shape[1]
#     height = input_image.shape[0]

#     # 이미지의 크기 조절
#     output_image = cv2.resize(input_image, dsize=(width // 4, height // 4))

#     # 복사된 이미지의 저장 경로
#     output_image_path = os.path.join(output_directory, filename)

#     # 이미지 저장
#     cv2.imwrite(output_image_path, output_image)

---

### shuffle

#### 정상

In [11]:
fine_train = pd.read_csv("C:/final_project/tomato_yolo_set/[라벨]토마토_정상_train(leaf).csv")
fine_train

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,1481,1345,2719,3233
1,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01_...,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,1480,1365,2710,3230
2,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,3095,930,4480,2980
3,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01_...,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,3105,940,4475,2985
4,V006_77_0_00_11_03_12_0_c01_20201013_0002_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,2130,205,4015,2655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4467,V006_77_0_00_11_03_13_0_c42_20201028_0056_S01_...,2020/10/28,4032,3024,77,0,NaN,0,11,3,13,0,521,618,3712,2532
4468,V006_77_0_00_11_03_13_0_c42_20201028_0057_S01.jpg,2020/10/28,4032,3024,77,0,NaN,0,11,3,13,0,106,860,3826,2554
4469,V006_77_0_00_11_03_13_0_c42_20201028_0057_S01_...,2020/10/28,4032,3024,77,0,NaN,0,11,3,13,0,110,868,3834,2546
4470,V006_77_0_00_11_03_13_0_c42_20201028_0058_S01.jpg,2020/10/28,4032,3024,77,0,NaN,0,11,3,13,0,554,824,3801,2462


In [12]:
fine_val = pd.read_csv("C:/final_project/tomato_yolo_set/[라벨]토마토_정상_val(leaf).csv")
fine_val

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_0_00_11_03_12_0_c22_20201130_0001_S01_...,2020/11/30,3024,4032,77,0,NaN,0,11,3,12,0,148,1129,2807,3338
1,V006_77_0_00_11_03_13_0_c04_20201223_0000_S01_...,2020/12/23,3024,3024,77,0,NaN,0,11,3,13,0,460,671,2640,1726
2,V006_77_0_00_11_03_13_0_c05_20201222_0000_S01_...,2020/12/22,3024,4032,77,0,NaN,0,11,3,13,0,803,821,2328,3085
3,V006_77_0_00_11_03_13_0_c05_20201222_0001_S01_...,2020/12/22,3024,4032,77,0,NaN,0,11,3,13,0,658,965,2948,3115
4,V006_77_0_00_11_03_13_0_c05_20201222_0002_S01_...,2020/12/22,3024,4032,77,0,NaN,0,11,3,13,0,555,1284,1808,3750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,V006_77_0_00_11_03_13_0_c23_20201201_0038_S01_...,2020/12/01,3024,4032,77,0,NaN,0,11,3,13,0,812,1118,2217,3604
1884,V006_77_0_00_11_03_13_0_c23_20201201_0039_S01_...,2020/12/01,3024,4032,77,0,NaN,0,11,3,13,0,665,1065,2087,3384
1885,V006_77_0_00_11_03_13_0_c23_20201201_0041_S01_...,2020/12/01,3024,4032,77,0,NaN,0,11,3,13,0,847,884,2393,3356
1886,V006_77_0_00_11_03_13_0_c23_20201201_0044_S01_...,2020/12/01,3024,4032,77,0,NaN,0,11,3,13,0,805,1483,2076,3421


In [21]:
fine_list = list(fine_train.image) + list(fine_val.image)

In [22]:
fine_list = pd.DataFrame(fine_list, columns=["image"])

In [24]:
fine_list.to_csv("C:/final_project/tomato_yolo_set/fine_list.csv", encoding="utf-8-sig", index=False)

#### 질병 1 잎 곰팡이

In [25]:
disease_train = pd.read_csv("C:/final_project/tomato_yolo_set/[라벨]토마토_질병_train.csv")
disease_train

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg,2020/10/05,4032,3024,77,1,4,18,11,3,13,1,908,916,3104,2312
1,V006_77_1_18_11_03_13_1_3248b_20201012_60.jpg,2020/10/12,4032,3024,77,1,4,18,11,3,13,1,1108,556,3356,2260
2,V006_77_1_18_11_03_13_1_3248b_20201012_61.jpg,2020/10/12,4032,3024,77,1,4,18,11,3,13,1,1152,772,3232,2420
3,V006_77_1_18_11_03_13_1_3248b_20201012_64.jpg,2020/10/12,4032,3024,77,1,4,18,11,3,13,1,540,544,2756,2224
4,V006_77_1_18_11_03_13_1_3248b_20201019_67.jpg,2020/10/19,4032,3024,77,1,4,18,11,3,13,1,648,852,3340,2524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,V006_77_1_19_11_03_12_3_9357q_20201130_28.jpeg,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,273,1004,3056,2244
1960,V006_77_1_19_11_03_12_3_9357q_20201130_29.jpeg,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,23,605,3575,2571
1961,V006_77_1_19_11_03_12_3_9357q_20201130_3.jpeg,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,256,398,3576,2924
1962,V006_77_1_19_11_03_12_3_9357q_20201130_7.jpeg,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,729,545,3735,2612


In [26]:
disease_val = pd.read_csv("C:/final_project/tomato_yolo_set/[라벨]토마토_질병_val.csv")
disease_val

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201012_63.jpg,2020/10/12,4032,3024,77,1,4,18,11,3,13,1,844,876,2944,2532
1,V006_77_1_18_11_03_13_1_7688w_20201111_215.jpg,2020/11/11,4032,3024,77,1,4,18,11,3,13,1,768,224,3508,2124
2,V006_77_1_18_11_03_13_1_7688w_20201112_225.jpg,2020/11/12,4032,3024,77,1,4,18,11,3,13,1,1228,792,3068,1968
3,V006_77_1_18_11_03_13_1_8419w_20200918_14.jpg,2020/09/18,4032,1908,77,1,4,18,11,3,13,1,834,0,3162,1713
4,V006_77_1_18_11_03_13_1_8419w_20200918_15.jpg,2020/09/18,4032,1908,77,1,4,18,11,3,13,1,888,0,3671,1908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,V006_77_1_19_11_03_12_3_7711b_20201125_52.jpeg,2020/11/25,4032,3024,77,1,17,19,11,3,12,3,1610,674,2202,1230
242,V006_77_1_19_11_03_12_3_7711b_20201125_57.jpeg,2020/11/25,4032,3024,77,1,17,19,11,3,12,3,1947,636,2375,1168
243,V006_77_1_19_11_03_12_3_7711b_20201125_9.jpeg,2020/11/25,4032,3024,77,1,17,19,11,3,12,3,1357,1228,2513,2268
244,V006_77_1_19_11_03_12_3_7711b_20201210_12.jpeg,2020/12/10,4032,3024,77,1,17,19,11,3,12,3,344,380,2520,2612


In [36]:
list(disease_train[disease_train["disease"].isin([18])].image)

['V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201012_60.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201012_61.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201012_64.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201019_67.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201026_19.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201102_26.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201116_40.jpg',
 'V006_77_1_18_11_03_13_1_3248b_20201130_49.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200916_1.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200916_6.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200918_12.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200922_20.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200922_21.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200922_24.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200922_25.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200923_32.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200923_33.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200923_34.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20200925_38.jpg',
 '

In [37]:
list(disease_val[disease_val["disease"].isin([18])].image)

['V006_77_1_18_11_03_13_1_3248b_20201012_63.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20201111_215.jpg',
 'V006_77_1_18_11_03_13_1_7688w_20201112_225.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20200918_14.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20200918_15.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20200918_23.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20201006_127.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20201006_130.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20201009_143.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20201011_154.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20201011_163.jpg',
 'V006_77_1_18_11_03_13_1_8419w_20201014_213.JPG',
 'V006_77_1_18_11_03_13_1_9008w_20200928_37.jpg',
 'V006_77_1_18_11_03_13_1_9008w_20200929_43.jpg',
 'V006_77_1_18_11_03_13_1_9008w_20201005_54.jpg',
 'V006_77_1_18_11_03_13_1_9008w_20201007_64.jpg',
 'V006_77_1_18_11_03_13_1_9008w_20201007_66.jpg',
 'V006_77_1_18_11_03_13_1_9008w_20201012_73.jpg',
 'V006_77_1_18_11_03_13_1_9008w_20201028_151.jpg',
 'V006_77_1_18_11_03_13_1_9008w_20201105_

In [38]:
aug_train = pd.read_csv("C:/final_project/tomato_yolo_set/[라벨]토마토_증강_train.csv")
aug_train

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,4032,3024,77,1,4,18,11,3,13,1,908,712,3104,2108
1,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,4032,3024,77,1,4,18,11,3,13,1,927,916,3124,2312
2,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,18,11,3,13,1,916,908,2312,3104
3,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,18,11,3,13,1,916,908,2312,3104
4,V006_77_1_18_11_03_13_1_3248b_20201005_57_a000...,2020/10/05,3024,4032,77,1,4,18,11,3,13,1,916,927,2312,3124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19005,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0005...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,644,438,3636,2526
19006,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0006...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,644,497,3636,2586
19007,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0007...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,644,497,3636,2586
19008,V006_77_1_19_11_03_12_3_9357q_20201130_8_a0008...,2020/11/30,3024,4032,77,1,17,19,11,3,12,3,497,396,2586,3388


In [41]:
aug_val = pd.read_csv("C:/final_project/tomato_yolo_set/[라벨]토마토_증강_val.csv")
aug_val

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201012_63_a000...,2020/10/12,3024,4032,77,1,4,18,11,3,13,1,876,1088,2532,3188
1,V006_77_1_18_11_03_13_1_3248b_20201012_63_a000...,2020/10/12,3024,4032,77,1,4,18,11,3,13,1,491,844,2147,2944
2,V006_77_1_18_11_03_13_1_3248b_20201012_63_a000...,2020/10/12,3024,4032,77,1,4,18,11,3,13,1,491,844,2147,2944
3,V006_77_1_18_11_03_13_1_3248b_20201012_63_a000...,2020/10/12,4032,3024,77,1,4,18,11,3,13,1,844,491,2944,2147
4,V006_77_1_18_11_03_13_1_3248b_20201012_63_a000...,2020/10/12,4032,3024,77,1,4,18,11,3,13,1,844,876,2944,2532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0005...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,608,698,3487,2686
2796,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0006...,2020/11/30,3024,4032,77,1,17,19,11,3,12,3,698,545,2686,3423
2797,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0007...,2020/11/30,3024,4032,77,1,17,19,11,3,12,3,698,545,2686,3423
2798,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0008...,2020/11/30,4032,3024,77,1,17,19,11,3,12,3,545,698,3423,2686


In [45]:
mold_list = list(disease_train[disease_train["disease"].isin([18])].image)+\
list(disease_val[disease_val["disease"].isin([18])].image)+\
list(aug_train[aug_train["disease"].isin([18])].image)+\
list(aug_val[aug_val["disease"].isin([18])].image)

In [50]:
mold_list_df = pd.DataFrame(mold_list, columns=["image"])

In [51]:
mold_list_df.to_csv("C:/final_project/tomato_yolo_set/mold_list.csv", encoding="utf-8-sig", index=False)

### TYLCV

In [47]:
tylcv_list = list(disease_train[disease_train["disease"].isin([19])].image)+\
list(disease_val[disease_val["disease"].isin([19])].image)+\
list(aug_train[aug_train["disease"].isin([19])].image)+\
list(aug_val[aug_val["disease"].isin([19])].image)

In [52]:
tylcv_list_df = pd.DataFrame(tylcv_list, columns=["image"])

In [54]:
tylcv_list_df.to_csv("C:/final_project/tomato_yolo_set/tylcv_list.csv", encoding="utf-8-sig", index=False)

### shuffle

## 사진 이동

In [10]:
fine_list = pd.read_csv("C:/final_project/tomato_yolo_set/fine_list.csv")
mold_list = pd.read_csv("C:/final_project/tomato_yolo_set/mold_list.csv")
tylcv_list = pd.read_csv("C:/final_project/tomato_yolo_set/tylcv_list.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/final_project/tomato_yolo_set/fine_list.csv'

In [68]:
fine = list(fine_list.image)
mold = list(mold_list.image)
tylcv = list(tylcv_list.image)

In [69]:
len(fine), len(mold), len(tylcv)

(6360, 10516, 13504)

In [11]:
fine = 6360
mold = 10516
tylcv = 13504

In [12]:
fine + mold + tylcv

30380

In [13]:
((fine*0.1) + (mold*0.1) + (tylcv*0.1))

3038.0

In [18]:
30380-3038-3038

24304

In [88]:
np.random.randint(0, 4)

0

In [89]:
import os
import shutil
import random

# 원본 폴더 경로
source_folder = "C:/final_project/tomato_yolo_shuffle/train/images"

# 대상 폴더 경로
target_folder = "C:/final_project/tomato_yolo_shuffle/test/images"

move_percentage = 0.1

# 파일 리스트
# file_list = fine
file_list = mold
# file_list = tylcv


# 개수 계산
move_count = int(len(file_list) * move_percentage)

# 랜덤
random_files = random.sample(file_list, move_count)
for file_name in random_files:
    # 파일의 전체 경로 생성
    source_file_path = os.path.join(source_folder, file_name)
    
    # 파일 존재 여부 확인
    if os.path.exists(source_file_path):
        # 대상 폴더에 파일 이동
        shutil.move(source_file_path, target_folder)
        print(f'{file_name}을(를) 이동했습니다.')
    else:
        print(f'{file_name}을(를) 찾을 수 없습니다.')

V006_77_1_18_11_03_13_3_8419w_20201009_135.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_3_3467b_20201005_17_a0003.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_9605w_20201116_238_a0005.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_9605w_20201118_247_a0000.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_8419w_20200918_22_a0002.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_9605w_20201102_186_a0009.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_3_3622w_20201007_69_a0003.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_3_9720w_20201016_93_a0009.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_3_3467b_20201015_27_a0006.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_9605w_20201109_216_a0000.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_9605w_20201005_79_a0001.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_7688w_20200916_6_a0008.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_9720w_20201013_78_a0001.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_7688w_20201020_111_a0004.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_9605w_20201009_92_a0002.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_9605w_20201111_22

In [ ]:
import os
import shutil
import random

# 원본 이미지 폴더 경로
source_image_folder = "C:/final_project/tomato_yolo_shuffle/train/images"
# 원본 텍스트 폴더 경로
source_text_folder = "C:/final_project/tomato_yolo_shuffle/train/labels"


# 대상 이미지 폴더 경로

# target_image_folder = "C:/final_project/tomato_yolo_shuffle/test/images"
target_image_folder = "C:/final_project/tomato_yolo_shuffle/val/images"

# 대상 텍스트 폴더 경로

# target_text_folder = "C:/final_project/tomato_yolo_shuffle/test/labels"
target_text_folder = "C:/final_project/tomato_yolo_shuffle/val/labels"


# 이동시킬 파일의 비율 (10%)
move_percentage = 0.1

# 원본 이미지 폴더의 파일 리스트 가져오기
image_file_list = os.listdir(source_image_folder)

# 이동시킬 파일 개수 계산
# move_count = int(len(image_file_list) * move_percentage)
move_count = int(30380 * move_percentage)

# 랜덤으로 파일 선택하여 이동
random_image_files = random.sample(image_file_list, move_count)
for image_file_name in random_image_files:
    # 이미지 파일의 전체 경로 생성
    source_image_path = os.path.join(source_image_folder, image_file_name)
    
    # 이미지 파일 존재 여부 확인
    if not os.path.exists(source_image_path):
        print(f'이미지 파일 {image_file_name}을(를) 찾을 수 없습니다.')
        continue

    # 이미지 파일의 이름을 기준으로 해당하는 텍스트 파일 경로 생성
    text_file_name = os.path.splitext(image_file_name)[0] + '.txt'
    source_text_path = os.path.join(source_text_folder, text_file_name)
    
    # 텍스트 파일 존재 여부 확인
    if not os.path.exists(source_text_path):
        print(f'텍스트 파일 {text_file_name}을(를) 찾을 수 없습니다.')
        continue
    
    # 대상 이미지 폴더에 이미지 파일 이동
    target_image_path = os.path.join(target_image_folder, image_file_name)
    shutil.move(source_image_path, target_image_path)
    print(f'{image_file_name}을(를) 이동했습니다.')
    
    # 대상 텍스트 폴더에 텍스트 파일 이동
    target_text_path = os.path.join(target_text_folder, text_file_name)
    shutil.move(source_text_path, target_text_path)
    print(f'{text_file_name}을(를) 이동했습니다.')


V006_77_1_19_11_03_12_2_2588q_20201130_25_a0000.jpeg을(를) 이동했습니다.
V006_77_1_19_11_03_12_2_2588q_20201130_25_a0000.txt을(를) 이동했습니다.
V006_77_1_19_11_03_12_2_2588q_20201208_5_a0003.jpeg을(를) 이동했습니다.
V006_77_1_19_11_03_12_2_2588q_20201208_5_a0003.txt을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_3467b_20201022_37_a0008.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_3467b_20201022_37_a0008.txt을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_7688w_20200925_40_a0000.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_7688w_20200925_40_a0000.txt을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_9720w_20201103_166_a0007.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_1_9720w_20201103_166_a0007.txt을(를) 이동했습니다.
V006_77_1_19_11_03_12_2_5952q_20201209_30_a0005.jpeg을(를) 이동했습니다.
V006_77_1_19_11_03_12_2_5952q_20201209_30_a0005.txt을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_9605w_20201127_291_a0008.jpg을(를) 이동했습니다.
V006_77_1_18_11_03_13_2_9605w_20201127_291_a0008.txt을(를) 이동했습니다.
V006_77_1_19_11_03_12_2_4997q_20201118_28_a0007.jpeg을(를) 이동했습니다.
V006_77_1_19_11_03_12_2_4997q_2020

In [167]:
import os
import shutil

file_list = move_out_list
# 이동시키고자 하는 파일들의 리스트


# 원본 폴더 경로
# source_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/train/images"
source_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/val/images"


# 대상 폴더 경로
# target_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/move_out/train_out"
target_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/move_out/val_out"

for file_name in file_list:
    # 파일의 전체 경로 생성
    source_file_path = os.path.join(source_folder, file_name)
    
    # 파일 존재 여부 확인
    if os.path.exists(source_file_path):
        # 대상 폴더에 파일 이동
        shutil.move(source_file_path, target_folder)
        print(f'{file_name}을(를) 이동했습니다.')
    else:
        print(f'{file_name}을(를) 찾을 수 없습니다.')

V006_77_0_00_11_01_13_0_c07_20201202_0068_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199.jpg을(를) 찾을 수 없습니다.
V006_77_1_19_11_03_12_1_4997q_20201130_22.jpeg을(를) 찾을 수 없습니다.
V006_77_1_19_11_03_12_3_4521q_20201209_21.jpeg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0000.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0001.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0002.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0003.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0004.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0005.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0006.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0007.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0008.jpg을(를) 찾을 수 없습니다.
V006_77_1_18_11_03_13_1_9008w_20201106_199_a0009.jpg을(를) 찾을 수 없습니다.
V006_77_1_19_11_03_12_3_4521q_20201209_21_a0000.jpeg을(를) 찾을 수 없습니다.